### IMPORTS

In [ ]:
import numpy as np
from Meshing.Meshing import *
from Fields.Fields import *
from Kernels.Kernels import *
from Solvers.Solvers import *
from Subchannel.FluidRelation import FluidRelation
from Subchannel.Channel import Channel
from Subchannel.Channel import ChannelInterface
from Aux.CSVObjects import *
from Aux.ReactorPhysicsObjects import *



### ST.ST. THERMAL HYDRAULICS

In [ ]:
_dt = 1e321
gravity = 9.81
Dh = 0.0158276279311347000000
perim = 0.07267978843558340
area = 2.572980283E-04
temp_tolerance = 1e-10
max_temp_iterations = 1000
nZones=25
L0 = 0.0
L1 = 67*2.54/100
fluid = FluidRelation(cp=1983, mu=0.00744, k=1.44, rho_0 = 2715.13, drho_dT=-0.513)
pressure_bc = 0.0
T_bc = 900.0
MDOT_MAX = 0.120101854
fric = 'type1'
heat_source = [0.0]*nZones # heat source in W/m3

upper_plenum_area = 1.065948974E-04 * 1.0
lower_plenum_area = 1.065948974E-04* 1.0
external_loop_area =6.810652553E-04* 1.0

# Change mdot based on csv values:
mdot_csv = load_csv('Data/mass_flow_startup.csv')
mdot_bc = MDOT_MAX / 100 * csv_interpolator(csv_df=mdot_csv, x_value=0.0, x_label='time', y_label='mdot')

#################
# CHANNEL SETUPS
#################
ch = Channel(gravity=gravity,
             Dh=Dh,
             area=area,
             temp_tolerance=temp_tolerance,
             max_temp_iterations=max_temp_iterations,
             nZones=nZones,
             L0=L0,
             L1=L1,
             fluid=fluid,
             pressure_bc=pressure_bc,
             T_bc=T_bc,
             mdot_bc=mdot_bc,
             fric=fric,
             heat_source=heat_source)

upper_plenum = Channel(gravity=gravity,
                       Dh=Dh,
                       area=upper_plenum_area,
                       temp_tolerance=temp_tolerance,
                       max_temp_iterations=max_temp_iterations,
                       nZones=nZones,
                       L0=0.0,
                       L1=1.0,
                       fluid=fluid,
                       pressure_bc=pressure_bc,
                       T_bc=1.0,
                       mdot_bc=1.0,
                       fric='none',
                       heat_source=0.0)

external_loop = Channel(gravity=gravity,
                       Dh=Dh,
                       area=external_loop_area,
                       temp_tolerance=temp_tolerance,
                       max_temp_iterations=max_temp_iterations,
                       nZones=nZones,
                       L0=0.0,
                       L1=1.0,
                       fluid=fluid,
                       pressure_bc=pressure_bc,
                       T_bc=1.0,
                       mdot_bc=1.0,
                       fric='none',
                       heat_source=0.0)

lower_plenum = Channel(gravity=gravity,
                       Dh=Dh,
                       area=lower_plenum_area,
                       temp_tolerance=temp_tolerance,
                       max_temp_iterations=max_temp_iterations,
                       nZones=nZones,
                       L0=0.0,
                       L1=1.0,
                       fluid=fluid,
                       pressure_bc=pressure_bc,
                       T_bc=1.0,
                       mdot_bc=1.0,
                       fric='none',
                       heat_source=0.0)

# Interfaces to handle data passing from one channel to another.
ch_to_up = ChannelInterface(ch1=ch, ch2=upper_plenum)
up_to_ex = ChannelInterface(ch1=upper_plenum, ch2=external_loop)
ex_to_lp = ChannelInterface(ch1=external_loop, ch2=lower_plenum)
lp_to_ch = ChannelInterface(ch1=lower_plenum, ch2=ch)

# Solution order - main channel -> plenum -> external loop -> lower plenum -> main channel
ch.solve_channel_TH(_dt=_dt)
ch_to_up.update_interface_conditions(tracer_bool=False, th_bool=True)

upper_plenum.solve_channel_TH(_dt=_dt)
up_to_ex.update_interface_conditions(tracer_bool=False, th_bool=True)

external_loop.solve_channel_TH(_dt=_dt)
ex_to_lp.update_interface_conditions(tracer_bool=False, th_bool=True)

lower_plenum.solve_channel_TH(_dt=_dt)
lp_to_ch.update_interface_conditions(tracer_bool=False, th_bool=True)

# End
print(ch.get_channel_residence_time())
print(upper_plenum.get_channel_residence_time())
print(external_loop.get_channel_residence_time())
print(lower_plenum.get_channel_residence_time())


### STEADY STATE TRACERS

In [ ]:
# source ratios
channel_ratio = 0.4178 + 0.3069 + 0.1279
lp_ratio = 0.0859 + 0.0226
up_ratio = 0.0389


tracer_names = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
initial_value_tracers = 0.0
scheme_tracers = 'upwind'
# OPENMC DNP DATA
# decay_consts = [0.01334, 0.03273, 0.1208, 0.3029, 0.8498, 2.854]
# beta = [0.000228, 0.001177, 0.001124, 0.002522, 0.001036, 0.000434]

# SINGH ET AL NONLINEAR DYNAMIC MODEL
decay_consts = [0.0124, 0.0305, 0.111, 0.301, 1.14, 3.01]
beta = [2.23e-4, 14.57e-4, 13.07e-4, 26.28e-4, 7.66e-4, 2.80e-4]

boundary = 'lower' # must be lower anyways
phi_tracer = 1.0
rho_tracer = 1.0

# MAKE SOURCES FOR FISSION SOURCE / IMPORTANCE WEIGHTING
src_function = np.cos(np.pi * ( np.array(ch.mesh.centroids) -L1/2) / (L1*1.0))
fsrc =   ScalarField(name='fsrc',   initial_value=src_function,    mesh=ch.mesh)
up_src = ScalarField(name='up_pre', initial_value=np.ones(nZones), mesh=upper_plenum.mesh)
lp_src = ScalarField(name='lp_pre', initial_value=np.ones(nZones), mesh=lower_plenum.mesh)

ch_src_int = fsrc.field_volume_integral()
up_src_int = up_src.field_volume_integral()
lp_src_int = lp_src.field_volume_integral()
total_int = ch_src_int + up_src_int + lp_src_int

fsrc.T = fsrc.T     * channel_ratio / ch_src_int
up_src.T = up_src.T * up_ratio / up_src_int
lp_src.T = lp_src.T * lp_ratio / lp_src_int

print("MAIN CHANNEL SRC FRAC", fsrc.field_volume_integral())
print("UPPER PLENUM SRC FRAC", up_src.field_volume_integral())
print("LOWER PLENUM SRC FRAC", lp_src.field_volume_integral())
print("TOTAL SRC FRAC", fsrc.field_volume_integral() + up_src.field_volume_integral() + lp_src.field_volume_integral())

# TRACER SETUP IN MAIN CHANNEL
for idx, name in enumerate(tracer_names):
  ch.add_tracer_to_channel(name=tracer_names[idx], initial_value=initial_value_tracers,
                          scheme=scheme_tracers, decay_const=decay_consts[idx],
                          boundary=boundary, phi=phi_tracer, rho=rho_tracer, source=fsrc, beta=beta[idx])

# TRACER SETUP IN UPPER PLENUM
for idx, name in enumerate(tracer_names):
  upper_plenum.add_tracer_to_channel(name=tracer_names[idx], initial_value=0.0,
                                     scheme=scheme_tracers, decay_const=decay_consts[idx],
                                     boundary=boundary, phi=phi_tracer,rho=rho_tracer,source=up_src, beta=beta[idx])

# TRACER SETUP IN EXTERNAL LOOP
for idx, name in enumerate(tracer_names):
  external_loop.add_tracer_to_channel(name=tracer_names[idx], initial_value=0.0,
                                     scheme=scheme_tracers, decay_const=decay_consts[idx],
                                     boundary=boundary, phi=phi_tracer,rho=rho_tracer,source=0.0, beta=beta[idx])

# TRACER SETUP IN LOWER PLENUM
for idx, name in enumerate(tracer_names):
  lower_plenum.add_tracer_to_channel(name=tracer_names[idx], initial_value=0.0,
                                     scheme=scheme_tracers, decay_const=decay_consts[idx],
                                     boundary=boundary, phi=phi_tracer,rho=rho_tracer,source=lp_src, beta=beta[idx])

# TRACER SOLVE LOOP
ch_inlet_value = phi_tracer
while True:
  ch.solve_all_tracers(_dt=_dt)
  ch_to_up.update_interface_conditions(tracer_bool=True, th_bool=False)

  upper_plenum.solve_all_tracers(_dt=_dt)
  up_to_ex.update_interface_conditions(tracer_bool=True, th_bool=False)

  external_loop.solve_all_tracers(_dt=_dt)
  ex_to_lp.update_interface_conditions(tracer_bool=True, th_bool=False)

  lower_plenum.solve_all_tracers(_dt=_dt)
  lp_to_ch.update_interface_conditions(tracer_bool=True,th_bool=False)

  diff = np.abs(ch_inlet_value - ch.tracers['c1'].T[0])
  print(diff)
  ch_inlet_value = ch.tracers['c1'].T[0]
  if diff < 1e-12:
    break




### RUN TRANSIENT

In [ ]:
# Time settings
Tstart = -2
Tend = 50
nsteps = 5000

# Timesteps
timesteps = np.linspace(Tstart, Tend, nsteps)

# Update old values in channel before starting simulation
ch.update_old_to_most_recent()
upper_plenum.update_old_to_most_recent()
external_loop.update_old_to_most_recent()
lower_plenum.update_old_to_most_recent()

# Transient solver
t_prev = -999999

# setup beff dict
beta_eff_dict = {}
beta_eff_dict_mc = {}

for t_new in timesteps:
  # COMPUTE NEW DELTA_T
  this_dt = t_new - t_prev

  # PRINT TIME INFORMATION
  print("NOW SOLVING AT TIME =", t_new, "| this_dt =", this_dt)

  # CHANGE MASS FLOW RATE AT CHANNEL INLET
  THIS_X_VALUE = t_new
  mdot_bc = MDOT_MAX / 100 * csv_interpolator(csv_df=mdot_csv, x_value=THIS_X_VALUE, x_label='time', y_label='mdot')
  ch.mdot_bc = mdot_bc

  # THERMAL HYDRAULIC TRANSIENT SOLUTION
  ch.solve_channel_TH(_dt=this_dt)
  ch_to_up.update_interface_conditions(tracer_bool=False, th_bool=True)

  upper_plenum.solve_channel_TH(_dt=this_dt)
  up_to_ex.update_interface_conditions(tracer_bool=False, th_bool=True)

  external_loop.solve_channel_TH(_dt=this_dt)
  ex_to_lp.update_interface_conditions(tracer_bool=False, th_bool=True)

  lower_plenum.solve_channel_TH(_dt=this_dt)
  lp_to_ch.update_interface_conditions(tracer_bool=False, th_bool=True)

  # TRACER TRANSIENT SOLUTION
  ch.solve_all_tracers(_dt=this_dt)
  ch_to_up.update_interface_conditions(tracer_bool=True, th_bool=False)

  upper_plenum.solve_all_tracers(_dt=this_dt)
  up_to_ex.update_interface_conditions(tracer_bool=True, th_bool=False)

  external_loop.solve_all_tracers(_dt=this_dt)
  ex_to_lp.update_interface_conditions(tracer_bool=True, th_bool=False)

  lower_plenum.solve_all_tracers(_dt=this_dt)
  lp_to_ch.update_interface_conditions(tracer_bool=True,th_bool=False)

  # SAVE BEFF INFORMATION
  this_beff, _ = compute_beff(channel=ch, weight=fsrc, names=tracer_names)
  beta_eff_dict[t_new] = this_beff

  this_beff_mc, _ = compute_beff_multichannel(channels=[ch, upper_plenum, lower_plenum], weights=[fsrc, up_src, lp_src], names=tracer_names)
  beta_eff_dict_mc[t_new] = this_beff_mc

  # Save channel data:
  ch.save_data(_t=t_new)
  upper_plenum.save_data(_t=t_new)
  external_loop.save_data(_t=t_new)
  lower_plenum.save_data(_t=t_new)

  ch.update_old_to_most_recent()
  upper_plenum.update_old_to_most_recent()
  external_loop.update_old_to_most_recent()
  lower_plenum.update_old_to_most_recent()

  t_prev = t_new

# End
print(ch.get_channel_residence_time())
print(upper_plenum.get_channel_residence_time())
print(external_loop.get_channel_residence_time())
print(lower_plenum.get_channel_residence_time())

In [ ]:
### EXP VALUES ###
msre_integral_worth = 'Data/msre_integral_rod_worth.csv'
msre_data = 'Data/msre_startup_datapoints.csv'

df = pd.read_csv(msre_integral_worth)
data = pd.read_csv(msre_data)

z = df['z'].values
rho = df[' rho'].values
pos = data['pos'].values
ornl_time_data = data['time'].values

values = np.interp(pos, z, rho)
base_insertion = values[0]
startup_base_insertion = values[0]

ornl_delta_rho_data = 1000*(values - base_insertion) # this is the data we calculated based on experimental values

# MAKE PLOT
plt.figure(figsize=(5,3))
plt.plot(ornl_time_data, ornl_delta_rho_data, 'ko', markerfacecolor='w', markersize=3)

### MY VALUES ###
x = np.array(list(beta_eff_dict.keys()))
y = beta_eff_dict[-2.0] - np.array(list(beta_eff_dict.values()))
y_MC = beta_eff_dict_mc[-2.0] - np.array(list(beta_eff_dict_mc.values()))
plt.plot(x, y*10**5, 'k-')
plt.plot(x, y_MC*10**5, 'k--')
# plt.plot(x[0::50], y[0::50]*10**5, 'ks', markerfacecolor='w')
# plt.plot(x[0::50], y[0::50]*10**5, 'k+', markerfacecolor='w')
plt.grid()
plt.xlim([-2,50])
plt.ylim([0,325])


